In [1]:
# Parameters
MODE = "gaussian"
DAYS = 1260
N_ITER = 500
METHODS = "momentum,sma_vix"
COST_BPS = "10,25,50"
MGMT_FEES = "0.01,0.02,0.03,0.04"
PERF_FEES = "0.05,0.10,0.20,0.25"
VOL_MULT = "1.0,1.5"
SEED = 2025
OUT_DIR = "mc/out"
PARAMS_PATH = "mc/params.json"
HIST_PATH = "mc/hist_returns.csv"


In [2]:
# Parameters (papermill will override these)
MODE        = "gaussian"          # "gaussian" or "block"
DAYS        = 252*5               # horizon
N_ITER      = 200                 # number of paths per scenario
METHODS     = "momentum,sma_vix"  # strategy set
COST_BPS    = "10,25,50"          # transaction-cost bps per weekly rebalance
MGMT_FEES   = "0.01,0.02,0.03,0.04"  # annual management fee
PERF_FEES   = "0.05,0.10,0.20,0.25"  # performance fee on excess vs SPY (charged yearly)
VOL_MULT    = "1.0,1.5"           # volatility scenarios
SEED        = 2025
OUT_DIR     = "mc/out"            # outputs: logs.txt, summary.csv, *.png
PARAMS_PATH = "mc/params.json"    # from fit_params.ipynb
HIST_PATH   = "mc/hist_returns.csv"


In [3]:
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

def load_params(path: str):
    with open(path, "r") as f:
        params = json.load(f)
    tickers = params["tickers"]
    mu  = np.array([params["mu_daily_pct"][t] for t in tickers])/100.0
    cov = np.array(params["cov_daily_pct2"])/(100.0**2)
    start_prices = np.array([params["start_prices"][t] for t in tickers], dtype=float)
    return tickers, mu, cov, start_prices

# --------- path generators ----------
def gaussian_paths(n_days, n_paths, mu, cov, start_prices, rng, vol_mult=1.0):
    n_assets = len(start_prices)
    cov_s = cov * (vol_mult**2)
    L = np.linalg.cholesky(cov_s + 1e-12*np.eye(n_assets))
    Z = rng.standard_normal(size=(n_paths, n_days, n_assets))
    R = mu + (Z @ L.T)                               # daily “log” returns (proportions)
    P = np.zeros((n_paths, n_days+1, n_assets))
    P[:,0,:] = start_prices
    for t in range(1, n_days+1):
        P[:,t,:] = P[:,t-1,:] * np.exp(R[:,t-1,:])
    return R, P

def block_bootstrap_paths(hist_csv: str, n_days: int, n_paths: int, rng, block=20, vol_mult=1.0):
    hist = pd.read_csv(hist_csv, parse_dates=["Date"]).set_index("Date")
    arr = hist.values/100.0
    n_hist, n_assets = arr.shape
    P = np.zeros((n_paths, n_days+1, n_assets))
    R = np.zeros((n_paths, n_days, n_assets))
    start_prices = np.full(n_assets, 100.0)
    P[:,0,:] = start_prices
    for i in range(n_paths):
        t = 0
        while t < n_days:
            s = rng.integers(0, max(1, n_hist - block))
            seg = arr[s:s+block]
            k = min(block, n_days - t)
            R[i, t:t+k, :] = seg[:k] * vol_mult
            t += k
        for t in range(1, n_days+1):
            P[i,t,:] = P[i,t-1,:] * np.exp(R[i,t-1,:])
    return R, P

# --------- metrics & fees ----------
def max_drawdown(equity):
    cummax = np.maximum.accumulate(equity)
    return float((equity/cummax - 1).min())

def annualize_return(daily_rets, dpy=252):
    g = np.prod(1 + daily_rets)
    years = daily_rets.size / dpy
    return g**(1/years) - 1.0

def sharpe_ratio(daily_rets, dpy=252, rf=0.0):
    mu = daily_rets.mean(); sd = daily_rets.std(ddof=1)
    return 0.0 if sd == 0 else ((mu - rf/dpy)/sd)*np.sqrt(dpy)

def alpha_beta(d_ret, b_ret, dpy=252):
    X = np.vstack([np.ones_like(b_ret), b_ret]).T
    coef, *_ = np.linalg.lstsq(X, d_ret, rcond=None)
    a, beta = coef
    alpha_ann = (1 + a)**dpy - 1
    return float(alpha_ann), float(beta)

def apply_mgmt_fee(daily_rets, mgmt_fee_annual, dpy=252):
    return daily_rets - (mgmt_fee_annual/dpy)

def apply_perf_fee_yearly(daily_rets, bench_daily, perf_fee, dpy=252):
    n = daily_rets.size
    out = daily_rets.copy()
    for y0 in range(0, n, dpy):
        y1 = min(y0 + dpy, n)
        rg = np.prod(1 + daily_rets[y0:y1]) - 1.0
        rb = np.prod(1 + bench_daily[y0:y1]) - 1.0
        fee = perf_fee * max(rg - rb, 0.0)
        if rg != -1.0:
            scale = (1 + rg - fee) / (1 + rg)
            out[y0:y1] = (1 + daily_rets[y0:y1])**scale - 1.0
    return out

# --------- strategy logic ----------
def ewma_vol(daily_ret, span=20):
    v = pd.Series(daily_ret).ewm(span=span, min_periods=span).std(bias=False).to_numpy()
    return v * np.sqrt(252) * 100.0

def compute_weights(method, prices, tickers, day_ix, lookback=60, vix_th=20.0):
    idx = {t:i for i,t in enumerate(tickers)}
    qqq = prices[:day_ix+1, idx["QQQ"]]
    gld = prices[:day_ix+1, idx["GLD"]]
    spy_ret = np.diff(np.log(prices[:day_ix+1, idx["SPY"]])) if day_ix>0 else np.array([0.0])

    if method == "momentum":
        if day_ix < lookback: return np.array([0.0, 1.0, 0.0])   # start in QQQ
        mom = qqq[-1]/qqq[-lookback] - 1.0
        return np.array([0.0, 1.0, 0.0]) if mom > 0 else np.array([0.0, 0.0, 1.0])

    if method == "sma_vix":
        vix_like = ewma_vol(spy_ret, span=20)
        vix_now = vix_like[-1] if vix_like.size else 0.0
        sma200 = gld[-200:].mean() if day_ix+1 >= 200 else gld.mean()
        hold_gld = (gld[-1] > sma200) or (vix_now >= vix_th)
        return np.array([0.0, 0.0, 1.0]) if hold_gld else np.array([0.0, 1.0, 0.0])

    raise ValueError(f"Unknown method: {method}")

def simulate_strategy(R, P, tickers, method, cost_bps=10, rebalance_freq=5):
    idx = {t:i for i,t in enumerate(tickers)}
    spy_ret = R[:, idx["SPY"]]
    n_days = R.shape[0]
    w_prev = np.array([0.0, 1.0, 0.0])  # start in QQQ
    daily_ret = np.zeros(n_days)
    turnover  = np.zeros(n_days)

    for t in range(n_days):
        if t % rebalance_freq == 0:
            w_new = compute_weights(method, P[:t+1,:], tickers, t)
            turn = np.abs(w_new - w_prev).sum()
            cost = turn * (cost_bps/10000.0)
            turnover[t] = turn
            w_prev = w_new.copy()
        r_today = np.exp(R[t,:]) - 1.0
        port_ret = (w_prev * r_today).sum()
        daily_ret[t] = port_ret - cost
    return daily_ret, spy_ret, turnover


In [4]:
# Parse list-like parameters
methods   = [m.strip() for m in METHODS.split(",") if m.strip()]
cost_list = [int(x) for x in COST_BPS.split(",")]
mgmt_list = [float(x) for x in MGMT_FEES.split(",")]
perf_list = [float(x) for x in PERF_FEES.split(",")]
vol_list  = [float(x) for x in VOL_MULT.split(",")]

out = Path(OUT_DIR); out.mkdir(parents=True, exist_ok=True)
logf = open(out/"logs.txt", "w", buffering=1)

tickers, mu, cov, start_prices = load_params(PARAMS_PATH)
assert {"SPY","QQQ","GLD"}.issubset(set(tickers)), "SPY/QQQ/GLD required."

rng_master = np.random.default_rng(SEED)
rows = []
sample_equities = []

total_runs = len(methods)*len(cost_list)*len(mgmt_list)*len(perf_list)*len(vol_list)*N_ITER
print(f"[MC] total runs ≈ {total_runs}", file=logf)

for method in methods:
    for cost in cost_list:
        for mgmt in mgmt_list:
            for perf in perf_list:
                for vm in vol_list:
                    rng = np.random.default_rng(rng_master.integers(1, 1_000_000_000))
                    for it in range(N_ITER):
                        if MODE == "gaussian":
                            R, P = gaussian_paths(DAYS, 1, mu, cov, start_prices, rng, vol_mult=vm)
                            R, P = R[0], P[0]
                        else:
                            R, P = block_bootstrap_paths(HIST_PATH, DAYS, 1, rng, block=20, vol_mult=vm)
                            R, P = R[0], P[0]

                        strat_ret, bench_ret, turn = simulate_strategy(R, P, tickers, method, cost_bps=cost)
                        r1 = apply_mgmt_fee(strat_ret, mgmt)
                        r2 = apply_perf_fee_yearly(r1, bench_ret, perf)

                        cagr  = annualize_return(r2)
                        vol   = r2.std(ddof=1)*np.sqrt(252)
                        sr    = sharpe_ratio(r2)
                        mdd   = max_drawdown(np.cumprod(1+r2))
                        alpha, beta = alpha_beta(r2, bench_ret)

                        rows.append({
                            "method": method, "cost_bps": cost, "mgmt_fee": mgmt, "perf_fee": perf,
                            "vol_mult": vm, "iter": it+1,
                            "CAGR_net": cagr, "Vol_net": vol, "Sharpe_net": sr, "MaxDD": mdd,
                            "Alpha_ann": alpha, "Beta": beta,
                            "Turnover_weekly_mean": float(np.mean(turn[turn>0])) if np.any(turn>0) else 0.0
                        })

                        if it < 3 and len(sample_equities) < 12:
                            eq = np.cumprod(1+r2)
                            be = np.cumprod(1+bench_ret)
                            sample_equities.append((method, cost, mgmt, perf, vm, eq, be))

# Save summary
df = pd.DataFrame(rows)
df.to_csv(out/"summary.csv", index=False)
print(f"[MC] saved summary: {out/'summary.csv'}", file=logf)

# Figures
if sample_equities:
    plt.figure(figsize=(9,5))
    for (method,cost,mgmt,perf,vm,eq,be) in sample_equities:
        lbl = f"{method}|cost{cost}|mgmt{mgmt}|perf{perf}|vm{vm}"
        plt.plot(eq, alpha=0.85, label=lbl)
    plt.plot(sample_equities[0][6], linestyle="--", alpha=0.85, label="SPY benchmark")
    plt.title("Sample Equity Curves (Net of Fees)")
    plt.xlabel("Days"); plt.ylabel("Equity (start=1.0)")
    plt.legend(fontsize=7, ncol=2); plt.tight_layout()
    plt.savefig(out/"equity_curves.png", dpi=160); plt.close()

if not df.empty:
    plt.figure(figsize=(8,5))
    for m in df["method"].unique():
        plt.hist(df[df["method"]==m]["CAGR_net"], bins=40, alpha=0.5, label=m)
    plt.title("CAGR Distribution (Net of Fees)")
    plt.xlabel("CAGR"); plt.ylabel("Frequency")
    plt.legend(); plt.tight_layout()
    plt.savefig(out/"cagr_dist.png", dpi=160); plt.close()

    plt.figure(figsize=(6,5))
    plt.scatter(df["Vol_net"], df["CAGR_net"], s=8, alpha=0.5)
    plt.xlabel("Volatility (annualized)"); plt.ylabel("CAGR (net)")
    plt.title("Risk–Return Scatter (Net of Fees)")
    plt.tight_layout()
    plt.savefig(out/"risk_return.png", dpi=160); plt.close()

logf.write("[MC] done.\n"); logf.close()
print(f"[MC] Finished. Outputs in {out}")


[MC] Finished. Outputs in mc/out
